In [103]:
using Pkg; Pkg.activate(".")
using HypergraphModularity

 Activating environment at `~/hypergraph_modularities_code/Project.toml`


In [104]:
# dataset = "email-Enron-full"
# dataset = "contact-primary-school-classes"
# dataset = "contact-high-school-classes"
# dataset = "congress-bills"
dataset = "walmart-trips"
# dataset = "TrivagoClickout"
kmax_ = 10

H, Z = read_hypergraph_data(dataset,kmax_, 2, true)
α = zeros(2*kmax_);

kmax = maximum(keys(H.E))
kmin = minimum(keys(H.E))

n = length(H.D)

function ω(p,α)
    k = p[2]
    δ = p[1]
    return ((1+(1-δ))*n)^α[k] / (n^α[k + kmax])
end

ω (generic function with 1 method)

In [105]:
# all or nothing
Ω = estimateΩEmpirically(H, Z; aggregator = p -> [length(p) == 1, sum(p)], bigNums = true);
Q_H = modularity(H, Z, Ω; α = nothing);
println(round(Q_H))

# number of groups
Ω = estimateΩEmpirically(H, Z; aggregator = p -> [length(p), sum(p)], bigNums = true);
Q_H = modularity(H, Z, Ω; α = nothing);
println(round(Q_H))


# smoothed parameterization
α0 = vcat(repeat([0.0], kmax), 1:kmax)

function ω(p, α)
    k = sum(p)
    return sum(p)/sum((p .* (1:length(p)).^α[k])) / n^(α[kmax+k]*k)
end
Ω = partitionIntensityFunction(ω, kmax);

### warmstart
α = learnParameters(H, Z, Ω, α0; n_iters = 100, amin = -10, amax = 20)

Q_H = modularity(H, Z, Ω; α = α)

println(round(Q_H))



-2.51109e+06
-2.499034e+06
-2.500949e+06


ω (generic function with 1 method)

In [112]:
# gamma = 1

ωᵢ = 2
ωₒ = 0.4

function ω(p, α)
    om = p[1] == 1 ? ωᵢ : ωₒ
    return om / (0.5*n)^sum(p)
end

Ω = allOrNothingIntensityFunction(ω, kmax)

Z = SuperNode_PPLouvain(H, Ω; α = zeros(2*kmax), verbose = false, Z0 = collect(1:n))

88860-element Array{Int64,1}:
     1
     2
     2
     2
     2
     2
     2
     2
     2
     2
     2
     3
     4
     ⋮
 22201
 22202
 22203
 22204
     2
     2
     2
 22205
 22206
 22207
     2
     2

In [114]:
unique(Z)

22207-element Array{Int64,1}:
     1
     2
     3
     4
     5
     6
     7
     8
     9
    10
    11
    12
    13
     ⋮
 22196
 22197
 22198
 22199
 22200
 22201
 22202
 22203
 22204
 22205
 22206
 22207